In [1]:
using JSON
using JuMP

poaDictAprPM = Dict{}()

using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM_ext

In [2]:
function POA_MA_Apr_PM_ext(day)
    
    #load OD pair-route incidence
    odPairRoute = readall("od_pair_route_incidence_MA_ext.json");
    odPairRoute = JSON.parse(odPairRoute);

    #load link-route incidence
    linkRoute = readall("link_route_incidence_MA_ext.json");
    linkRoute = JSON.parse(linkRoute);

    #load OD pair labels
    odPairLabel = readall("od_pair_label_dict_MA_refined_ext.json");
    odPairLabel = JSON.parse(odPairLabel);

    odPairLabel_ = readall("od_pair_label_dict__MA_refined_ext.json");
    odPairLabel_ = JSON.parse(odPairLabel_);

    #load link labels
    linkLabel = readall("link_label_dict_MA_ext.json");
    linkLabel = JSON.parse(linkLabel);

    linkLabel_ = readall("link_label_dict_MA__ext.json");
    linkLabel_ = JSON.parse(linkLabel_);

    #load node-link incidence
    nodeLink = readall("node_link_incidence_MA_ext.json");
    nodeLink = JSON.parse(nodeLink);

    include("load_network_uni-class.jl")

    ta_data = load_ta_network("MA");

    capacity = ta_data.capacity;
    free_flow_time = ta_data.free_flow_time;

    demandsDict = readall("../08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_PM/results/demandsDictFixed$(day).json");
    demandsDict = JSON.parse(demandsDict);

    # demands = demandsDict["$(length(demandsDict)-1)"]

    demands = demandsDict

    odPairLabel_;

    demandsVec = zeros(462)

    for i = 1:length(demandsVec)
        demandsVec[i] = demands["($(odPairLabel_["$i"][1]),$(odPairLabel_["$i"][2]))"]
    end

    demandsVec

    for key=keys(odPairRoute)
        if contains(key, "462-")
            println(key)
        end
    end

    linkRoute;

    deg = 6
    c = 0.4
    lam = 100.0

    coeffs_dict_Apr_PM_ = readall("../temp_files/coeffs_dict_Apr_PM.json")
    coeffs_dict_Apr_PM_ = JSON.parse(coeffs_dict_Apr_PM_)
    fcoeffs = coeffs_dict_Apr_PM_["($(deg),$(c),$(lam),1)"]

    # fcoeffs = [1, 0, 0, 0, .15]

    # m = Model(solver=GurobiSolver(OutputFlag=false))
    m = Model()

    numLinks = 74
    numRoute = 4614
    numOD = 462

    @defVar(m, linkFlow[1:numLinks])
    @defVar(m, pathFlow[1:numRoute])

    pathFlowSum = Dict()

    for i=1:numOD
        pathFlowSum[i] = 0
        for j=1:numRoute
            if "$(i)-$(j)" in keys(odPairRoute)
                pathFlowSum[i] += pathFlow[j]
            end
        end
        @addConstraint(m, pathFlowSum[i] == demandsVec[i])
    end

    pathFlowLinkSum = Dict()

    for a=1:numLinks
        pathFlowLinkSum[a] = 0
        for j=1:numRoute
            if "$(a)-$(j)" in keys(linkRoute)
                pathFlowLinkSum[a] += pathFlow[j]
            end
        end
        @addConstraint(m, pathFlowLinkSum[a] == linkFlow[a])
    end

    for j=1:numRoute
        @addConstraint(m, pathFlow[j] >= 0)
    end

    @defNLExpr(f, sum{free_flow_time[a] * fcoeffs[1] * linkFlow[a] + 
        free_flow_time[a] * fcoeffs[2] * linkFlow[a]^2 / capacity[a] +
        free_flow_time[a] * fcoeffs[3] * linkFlow[a]^3 / capacity[a]^2 +
        free_flow_time[a] * fcoeffs[4] * linkFlow[a]^4 / capacity[a]^3 +
        free_flow_time[a] * fcoeffs[5] * linkFlow[a]^5 / capacity[a]^4 +
        free_flow_time[a] * fcoeffs[6] * linkFlow[a]^6 / capacity[a]^5 +
        free_flow_time[a] * fcoeffs[7] * linkFlow[a]^7 / capacity[a]^6, a = 1:numLinks})

    @setNLObjective(m, Min, f)

    solve(m)

    println(getValue(linkFlow))

    getObjectiveValue(m)

    flows = Dict{(Int64,Int64),Float64}()

    for i = 1:length(ta_data.start_node)
        key = (ta_data.start_node[i], ta_data.end_node[i])
        flows[key] = getValue(linkFlow)[i]
    end

    flows

    # getValue(linkFlow)

    # getObjectiveValue(m)

    flow_user = GLS_Apr_weekday_PM_ext.x_

    function socialObj(linkFlowVec)
        objVal = sum([free_flow_time[a] * fcoeffs[1] * linkFlowVec[a] + 
            free_flow_time[a] * fcoeffs[2] * linkFlowVec[a]^2 / capacity[a] + 
            free_flow_time[a] * fcoeffs[3] * linkFlowVec[a]^3 / capacity[a]^2 + 
            free_flow_time[a] * fcoeffs[4] * linkFlowVec[a]^4 / capacity[a]^3 + 
            free_flow_time[a] * fcoeffs[5] * linkFlowVec[a]^5 / capacity[a]^4 + 
            free_flow_time[a] * fcoeffs[6] * linkFlowVec[a]^6 / capacity[a]^5 + 
            free_flow_time[a] * fcoeffs[7] * linkFlowVec[a]^7 / capacity[a]^6 for a = 1:numLinks])
        return objVal
    end
    
    println(socialObj(flow_user[:, day])/getObjectiveValue(m))
    
    return socialObj(flow_user[:, day])/getObjectiveValue(m)
end

POA_MA_Apr_PM_ext (generic function with 1 method)

In [3]:
function POA_MA_Apr_PM_ext_alt(day)
    
    #load OD pair-route incidence
    odPairRoute = readall("od_pair_route_incidence_MA_ext.json");
    odPairRoute = JSON.parse(odPairRoute);

    #load link-route incidence
    linkRoute = readall("link_route_incidence_MA_ext.json");
    linkRoute = JSON.parse(linkRoute);

    #load OD pair labels
    odPairLabel = readall("od_pair_label_dict_MA_refined_ext.json");
    odPairLabel = JSON.parse(odPairLabel);

    odPairLabel_ = readall("od_pair_label_dict__MA_refined_ext.json");
    odPairLabel_ = JSON.parse(odPairLabel_);

    #load link labels
    linkLabel = readall("link_label_dict_MA_ext.json");
    linkLabel = JSON.parse(linkLabel);

    linkLabel_ = readall("link_label_dict_MA__ext.json");
    linkLabel_ = JSON.parse(linkLabel_);

    #load node-link incidence
    nodeLink = readall("node_link_incidence_MA_ext.json");
    nodeLink = JSON.parse(nodeLink);

    include("load_network_uni-class.jl")

    ta_data = load_ta_network("MA");

    capacity = ta_data.capacity;
    free_flow_time = ta_data.free_flow_time;

    demandsDict = readall("../08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_PM/results/demandsDictFixed$(day).json");
    demandsDict = JSON.parse(demandsDict);

    # demands = demandsDict["$(length(demandsDict)-1)"]

    demands = demandsDict
    
    demands_ = Dict{}()

    for key in keys(demands)
        key_ = (int(split(split(key, ',')[1], '(')[2]),int(split(split(key, ',')[2], ')')[1]))
        demands_[key_] = demands[key]
    end

    odPairLabel_;

    demandsVec = zeros(462)

    for i = 1:length(demandsVec)
        demandsVec[i] = demands["($(odPairLabel_["$i"][1]),$(odPairLabel_["$i"][2]))"]
    end

    demandsVec

    for key=keys(odPairRoute)
        if contains(key, "462-")
            println(key)
        end
    end

    linkRoute;

    deg = 6
    c = 0.4
    lam = 100.0

    coeffs_dict_Apr_PM_ = readall("../temp_files/coeffs_dict_Apr_PM.json")
    coeffs_dict_Apr_PM_ = JSON.parse(coeffs_dict_Apr_PM_)
    fcoeffs = coeffs_dict_Apr_PM_["($(deg),$(c),$(lam),1)"]

    # fcoeffs = [1, 0, 0, 0, .15]
    
    # m = Model(solver=GurobiSolver(OutputFlag=false))
    m = Model()

    numLinks = 74  # number of links
    numNodes = 22  # number of nodes

    @defVar(m, linkFlow[1:numLinks])

    @defVar(m, odLinkFlow[keys(demands_), 1:numLinks])
    for (s,t) = keys(demands_)
        for k = 1:numLinks
            @addConstraint(m, odLinkFlow[(s,t), k] >= 0)
        end
    end

    for k = 1:numLinks
        linkFlowPartial = sum([odLinkFlow[(s,t), k] for (s,t) = keys(demands_)])
        @addConstraint(m, linkFlowPartial == linkFlow[k])
    end

    for l = 1:numNodes
        for (s,t) = keys(demands_)
            if s != t
                odLinkFlowPartial = sum([nodeLink["$(l-1)-$(k-1)"] * odLinkFlow[(s,t), k] for k = 1:numLinks])
                if (l == s)
                    @addConstraint(m, odLinkFlowPartial + demands_[(s,t)] == 0)
                elseif (l == t)
                    @addConstraint(m, odLinkFlowPartial - demands_[(s,t)] == 0)
                else
                    @addConstraint(m, odLinkFlowPartial == 0)
                end
            end
        end
    end

    @defNLExpr(f, sum{free_flow_time[a] * fcoeffs[1] * linkFlow[a] + 
        free_flow_time[a] * fcoeffs[2] * linkFlow[a]^2 / capacity[a] +
        free_flow_time[a] * fcoeffs[3] * linkFlow[a]^3 / capacity[a]^2 +
        free_flow_time[a] * fcoeffs[4] * linkFlow[a]^4 / capacity[a]^3 +
        free_flow_time[a] * fcoeffs[5] * linkFlow[a]^5 / capacity[a]^4 +
        free_flow_time[a] * fcoeffs[6] * linkFlow[a]^6 / capacity[a]^5 +
        free_flow_time[a] * fcoeffs[7] * linkFlow[a]^7 / capacity[a]^6, a = 1:numLinks})

    @setNLObjective(m, Min, f)

    solve(m)

    println(getValue(linkFlow))

    getObjectiveValue(m)

    flows = Dict{(Int64,Int64),Float64}()

    for i = 1:length(ta_data.start_node)
        key = (ta_data.start_node[i], ta_data.end_node[i])
        flows[key] = getValue(linkFlow)[i]
    end

    flows

    # getValue(linkFlow)

    # getObjectiveValue(m)

    flow_user = GLS_Apr_weekday_PM_ext.x_

    function socialObj(linkFlowVec)
        objVal = sum([free_flow_time[a] * fcoeffs[1] * linkFlowVec[a] + 
            free_flow_time[a] * fcoeffs[2] * linkFlowVec[a]^2 / capacity[a] + 
            free_flow_time[a] * fcoeffs[3] * linkFlowVec[a]^3 / capacity[a]^2 + 
            free_flow_time[a] * fcoeffs[4] * linkFlowVec[a]^4 / capacity[a]^3 + 
            free_flow_time[a] * fcoeffs[5] * linkFlowVec[a]^5 / capacity[a]^4 + 
            free_flow_time[a] * fcoeffs[6] * linkFlowVec[a]^6 / capacity[a]^5 + 
            free_flow_time[a] * fcoeffs[7] * linkFlowVec[a]^7 / capacity[a]^6 for a = 1:numLinks])
        return objVal
    end
    
    println(socialObj(flow_user[:, day])/getObjectiveValue(m))
    
    return socialObj(flow_user[:, day])/getObjectiveValue(m)
end

POA_MA_Apr_PM_ext_alt (generic function with 1 method)

In [4]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]
# week_day_Apr_list = [11]

21-element Array{Int64,1}:
  2
  3
  4
  5
  6
  9
 10
 11
 12
 13
 16
 17
 18
 19
 20
 23
 24
 25
 26
 27
 30

In [5]:
# for day in week_day_Apr_list
#     poaDictAprPM[day] = POA_MA_Apr_PM_ext(day)
# end

In [6]:
for day in week_day_Apr_list
    poaDictAprPM[day] = POA_MA_Apr_PM_ext_alt(day)
end

462-329

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   104266
Number of nonzeros in inequality constraint Jacobian.:    35816
Number of nonzeros in Lagrangian Hessian.............:       74

Total number of variables............................:    35890
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number

In [9]:
outfile = open("./results/poaDictAprPM.json", "w")

JSON.print(outfile, poaDictAprPM)

close(outfile)

In [10]:
poaDictAprPM

Dict{Any,Any} with 21 entries:
  30 => 1.267194009338036
  2  => 1.4441846188325576
  16 => 1.3537046311650884
  11 => 1.836930827537202
  26 => 1.0423992161670523
  9  => 2.2943822461188668
  25 => 1.058404272657658
  10 => 2.5019123671580488
  19 => 1.2736398101556838
  17 => 1.7082395837402393
  6  => 1.3547416804356778
  24 => 1.053911210092728
  4  => 1.3658129794128762
  3  => 1.5543454795803051
  5  => 1.4411424098590755
  20 => 1.1532539167180922
  23 => 1.1257397579126427
  13 => 1.8335406606320561
  27 => 1.4163482779322276
  12 => 2.4226143635980018
  18 => 1.4528228929624722